In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import torch
from skimage import measure
from tqdm import tqdm, trange

from core.dataset import ScanNet
from core.integrate import ScalableTSDFVolume

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
dataset = ScanNet('/home/quanta/Datasets/ScanNet/')
scan_id = 'scene0000_00'
id = dataset.scan_id_list.index(scan_id)
single_instance = dataset[id]['scan_dataset']

In [4]:
tsdf_device = 'cuda:1'
tsdf_volume = ScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=tsdf_device,
)

In [5]:
save_dir = '/home/quanta/Experiments/feature-instance-fusion/scannet_scene0000_00/'
os.makedirs(save_dir, exist_ok=True)

In [6]:
indent = 1
for idx in trange(0, len(single_instance), indent):
# for idx in trange(0, 500, indent):
    inputs = single_instance.get_torch_tensor(idx, device=tsdf_device, keys={"depth", "depth_intr", "pose"})
    tsdf_volume.integrate_tsdf(
        depth=inputs["depth"],
        depth_intr=inputs["depth_intr"],
        cam_pose=inputs["pose"],
    )

tsdf_volume.prune_voxel()
tsdf_volume.save(save_dir + 'tsdf/tsdf_vol.pt')

# tsdf_volume.load(save_dir + 'tsdf/tsdf_vol.pt')

100%|██████████| 5578/5578 [06:04<00:00, 15.31it/s]


In [ ]:
verts, faces = tsdf_volume.extract_mesh()
np.save(save_dir + 'tsdf/verts.npy', verts)
np.save(save_dir + 'tsdf/faces.npy', faces)

In [ ]:
import open3d as o3d

draw = o3d.visualization.EV.draw
mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(verts),
    triangles=o3d.utility.Vector3iVector(faces),
)

In [ ]:
mesh.compute_vertex_normals()

In [ ]:
draw([mesh])